<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter6_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6-5 Model Training Using TPU

It only requires six additional of code when training model using TPU in Google Colab.

In Colab notebook, choose TPU in Edit -> Notebook Settings -> Hardware Accelerator.

Note: the following code only executes on Colab

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import *

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.15.0


## 1. Data Preparation

In [2]:
MAX_LEN = 300
BATCH_SIZE = 32

(x_train, y_train), (x_test, y_test) = datasets.reuters.load_data()
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=MAX_LEN)

MAX_WORDS = x_train.max() + 1
CAT_NUM = y_train.max() + 1

2110848/2110848 [==============================] - 0s 0us/step


In [3]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(buffer_size=1000).batch(BATCH_SIZE) \
    .prefetch(tf.data.experimental.AUTOTUNE).cache()

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .shuffle(buffer_size=1000).batch(BATCH_SIZE) \
    .prefetch(tf.data.experimental.AUTOTUNE).cache()

## 2. Model Defining

In [10]:
tf.keras.backend.clear_session()

def create_model():
  model = models.Sequential()

  model.add(layers.Embedding(MAX_WORDS, 7, input_length=MAX_LEN))
  model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
  model.add(layers.MaxPool1D(2))
  model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
  model.add(layers.MaxPool1D(2))
  model.add(layers.Flatten())
  model.add(layers.Dense(CAT_NUM, activation='softmax'))

  return (model)

def compile_model(model):
  model.compile(optimizer=optimizers.Nadam(),
                loss=losses.SparseCategoricalCrossentropy(),
                metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseTopKCategoricalAccuracy(5)])

  return (model)

## 3. Model Training

In [11]:
# Here is the 6 additional lines of code
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
  model = create_model()
  model.summary()
  model = compile_model(model)

KeyError: 'COLAB_TPU_ADDR'

In [12]:
  model = create_model()
  model.summary()
  model = compile_model(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 7)            216874    
                                                                 
 conv1d (Conv1D)             (None, 296, 64)           2304      
                                                                 
 max_pooling1d (MaxPooling1  (None, 148, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 2336)              0

In [13]:
history = model.fit(ds_train, validation_data=ds_test, epochs=10)

Epoch 1/10
281/281 [==============================] - 4s 11ms/step - loss: 2.0044 - sparse_categorical_accuracy: 0.4728 - sparse_top_k_categorical_accuracy: 0.7453 - val_loss: 1.6474 - val_sparse_categorical_accuracy: 0.5744 - val_sparse_top_k_categorical_accuracy: 0.7627
Epoch 2/10
281/281 [==============================] - 3s 9ms/step - loss: 1.4314 - sparse_categorical_accuracy: 0.6300 - sparse_top_k_categorical_accuracy: 0.8024 - val_loss: 1.5113 - val_sparse_categorical_accuracy: 0.6309 - val_sparse_top_k_categorical_accuracy: 0.7912
Epoch 3/10
281/281 [==============================] - 3s 9ms/step - loss: 1.0958 - sparse_categorical_accuracy: 0.7118 - sparse_top_k_categorical_accuracy: 0.8692 - val_loss: 1.5325 - val_sparse_categorical_accuracy: 0.6425 - val_sparse_top_k_categorical_accuracy: 0.8148
Epoch 4/10
281/281 [==============================] - 3s 9ms/step - loss: 0.7970 - sparse_categorical_accuracy: 0.7951 - sparse_top_k_categorical_accuracy: 0.9310 - val_loss: 1.7679 -